In [78]:
# load libraries
import pandas as pd
from datetime import datetime
from datetime import timedelta
import math
from sklearn import ensemble
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

In [79]:
# This section contains the functions of data loading and feature engineering


# load the data and aggragate it on 15 minute interval
def load_data(myfile='../data/logins2.json'):
    df = pd.read_json(myfile)
    
    # create a pandas Series with timestamp as index
    ts_series = pd.Series(1,index=df.login_time)
    
    # aggregate the count on 15 minute interval
    data = ts_series.resample('15T').sum()
    data = pd.DataFrame({'login_time':data.index,'count':data.values})
    
    return data

# create all the features

def create_features(data):
    data['day_category'] = data.login_time.apply(lambda x: str(x)[-8:-3])
    data['day_numeric'] = data.day_category.apply(lambda x: create_time_numeric(x))
    data['day_cos'] = data.day_numeric.apply(lambda x: math.cos(2*math.pi*x))
    data['day_sin'] = data.day_numeric.apply(lambda x: math.sin(2*math.pi*x))
    data['day_of_week'] = data.login_time.apply(lambda x: x.weekday())
    data['week_numeric'] = data.apply(lambda row: (row['day_of_week'] + row['day_numeric'])/7, axis=1)
    data['week_cos'] = data.week_numeric.apply(lambda x: math.cos(2*math.pi*x))
    data['week_sin'] = data.week_numeric.apply(lambda x: math.sin(2*math.pi*x))
    data['weekend'] = data.day_of_week.apply(lambda x: 1 if x >= 5 else 0)
    
    return data

# After all the features are created in the dataframe
# this function extracts and presents them as a 2d list
# In the case of preparing the data for future predictions,
# y is returned as None as it is not known at the time of 
# prediction
def extract_features(data):
    # return arrays
    X = data[['day_numeric','day_cos','day_sin','week_numeric','week_cos','week_sin','weekend']].values
    if 'count' in data.columns:
        y = data['count'].values
    else:
        y = None
    return X, y
    
# helper function for create_time_numeric
def create_numeric_minute(minute_str):
    if minute_str == '00':
        return 0.25*0 + 0.25/2
    elif minute_str == '15':
        return 0.25*1 + 0.25/2
    elif minute_str == '30':
        return 0.25*2 + 0.25/2
    elif minute_str == '45':
        return 0.25*3 + 0.25/2

# helper function for create_features
def create_time_numeric(time_category):
    h,m = time_category.split(':')
    h_num = float(h)
    m_num = create_numeric_minute(m)
    
    time_numeric = (h_num + m_num)/24
    
    return time_numeric

# combine functions in this cell into one
def load_and_preprocess():
    data = load_data()
    data = create_features(data)
    X,y = extract_features(data)
    return X, y

In [80]:
# show what we get with these functions

X,y = load_and_preprocess()
print(X[:2])
print(y[:2])

[[ 0.00520833  0.99946459  0.03271908  0.57217262 -0.89893063 -0.438091
   0.        ]
 [ 0.015625    0.99518473  0.09801714  0.57366071 -0.89479525 -0.44647671
   0.        ]]
[3 3]


In [81]:
# Set up sliding window cross validation and list the values of hyperparameters

# sliding window CV.
# The list contains of tuples of indices corressponding to specific login time points
# 1st tuple: (Jan.1 to Mar. 27 2:45pm) as training and (Mar. 27 3pm to Mar 28 2:45pm) as test
# 2nd tuple: (Mar.27 3pm to Jun. 26 2:45pm) as training and (Jun. 26 3pm to Jun. 27 2:45pm) as test
# 3rd tuple: (Jun. 26 3pm to Aug. 21 2:45pm) as training and (Aug. 21 3pm to Aug. 22 2:45pm) as test

cv=[(list(range(0,8220)),list(range(8220,8316))), (list(range(8220,16956)),list(range(16956,17052))),\
   (list(range(16956,22332)),list(range(22332,22428)))]

# list of hyperparameters to tune.
grid_params = [{'n_estimators':[300,400,500],'max_depth':[4,5,6],'min_samples_split':[2],\
              'learning_rate':[0.01,0.05,0.1],'loss':['ls']}]

In [82]:
# define functions to run grid search and ts cv
# for gradient boosting


# use RMSE as the loss function 
# Define it by myself
# the purpose is just to use square root
# as sklearn uses MSE.
# The same result can be acheived through 'loss':ls
# in GradientBoostingRegressor module
def RMSE(y_true,y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = mse**(1/2)
    #print('RMSE: %2.3f' % rmse)
    return rmse

# make it useable for sklearn module
my_scorer = make_scorer(RMSE, greater_is_better=True)

# run cv with time series data (split is custom and time sequence is maintained)
def run_gbm_timeseries_cv(cv,grid_params,X,y):
    gbm_cv = GridSearchCV(ensemble.GradientBoostingRegressor(),grid_params,cv=cv,scoring=my_scorer)
    gbm_cv.fit(X,y)
    print('Parameters of the best model: ', gbm_cv.best_params_)
    return gbm_cv

# after the cv, fit the best model and get training RMSE
def fit_best_gbm(gbm_cv,X,y):
    gbm_best = ensemble.GradientBoostingRegressor(**gbm_cv.best_params_)
    gbm_best.fit(X,y)
    RMSE(y,gbm_best.predict(X))
    print('RMSE of the entire data: ', RMSE(y,gbm_best.predict(X)))
    return gbm_best

# To predict the future, all the features need to be created based on time.
# Since the features are only time-based, we can produce without any external reference.
# This approach is very similar to simulation
def create_data_for_prediction(h=4): 
    data_future = pd.DataFrame({'login_time':[datetime(2010,8,28,15,0,0) + timedelta(minutes=15*i) for i in range(h)]})
    data_future = create_features(data_future)
    X_future,y_future = extract_features(data_future)
    return X_future,y_future

# Put all the gbm part of the functions into one
def fit_gbm_model(cv,grid_params):
    X,y = load_and_preprocess()
    gbm_cv = run_gbm_timeseries_cv(cv,grid_params,X,y)
    gbm_best = fit_best_gbm(gbm_cv,X[:22332],y[:22332]) # use the most recent 2 month, just to be on par with Fourier ARIMA prediction
    X_future,y_future = create_data_for_prediction()
    pred = gbm_best.predict(X_future)
    print("predictions for the next one hour from 3pm Aug. 28: ", pred)

In [83]:
fit_gbm_model(cv,grid_params)

Parameters of the best model:  {'learning_rate': 0.1, 'loss': 'ls', 'max_depth': 6, 'min_samples_split': 2, 'n_estimators': 500}
RMSE of the entire data:  2.119209828836946
predictions for the next one hour from 3pm Aug. 28:  [4.79953012 5.51202081 4.90260041 4.67850591]
